### Tensor Flow 2-11 notebook

Notes: 
- Notebook should be running with an Nvidia GPU to for top performance
- load_img will load a nparray with x, y, 3 color dims.  
- If we're just processing one image we will need to np.expand_dims(image, axies=0) to get 1, x, y, 3. The library expects N images in the first dim
- should set random seed on tf.random.set_seed(1) for reproduceability 
- 

last change 3/6/2025

In [ ]:
# import the libraries for training, testing, validation
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import imagenet_utils  # will decode predictions out of the model into a 4 dim array of N (image num), imageID, label, probability result[0] would be the set of results for image one
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img  # will load img and reshape, usage is load_img(image_name_loc, target_size=input_shape)
from tensorflow.keras.utils import plot_model  # Note: usage syntax is plot_model(instantied_model, to_file='', show_shapes=True)
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds  # For loading datasets from GCS

# import all model architectures
from tensorflow.keras.applications import MobileNetV2, MobileNetV3Large, MobileNetV3Small
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB7
from tensorflow.keras.applications import InceptionV3


import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import time


In [ ]:
# define a dictionary for model config and experiment tracking.... 
model_input_variables = {
    "EfficientNetB0": {
        "input_shape": (224, 224, 3),
        "batch_size": 32,
        "epochs": 10,
        "learning_rate": 0.0001,
    },
    "MobileNetV2": {
        "input_shape": (224, 224, 3),
        "batch_size": 32,
        "epochs": 10,
        "learning_rate": 0.0001,
    },
     "MobileNetV3Large": { #MobileNetV3 Large will be used to compare accuracy loss with small, prob not a candidate for rasp pi running
        "input_shape": (224, 224, 3),
        "batch_size": 32,
        "epochs": 10,
        "learning_rate": 0.0001,
    },
    "MobileNetV3Small":{ 
        "input_shape": (224, 224, 3),
        "batch_size": 32,
        "epochs": 10,
        "learning_rate": 0.0001,
    },
    "InceptionV3": {
        "input_shape": (299, 299, 3),  # InceptionV3 typically uses 299x299
        "batch_size": 32,
        "epochs": 10,
        "learning_rate": 0.0001,
    },
    "EfficientNetB7":{
        "input_shape": (600, 600, 3),  # big values!
        "batch_size": 16, # this is a larger model and the norm seems to be smaller batch sizes for reduced memory use
        "epochs": 10,
        "learning_rate": 0.0001,
    }
}

print(f'EfficientNetB0: {model_input_variables["EfficientNetB0"]}')
print(f'MobileNetV2: {model_input_variables["MobileNetV2"]}')
print(f'MobileNetV3Large: {model_input_variables["MobileNetV3Large"]}')
print(f'MobileNetV3Small: {model_input_variables["MobileNetV3Small"]}')
print(f'InceptionV3: {model_input_variables["InceptionV3"]}')
print(f'EfficientNetB7: {model_input_variables["EfficientNetB7"]}')

In [ ]:
# Experiment Configuration and globals
gcs_bucket = 'nabirds_filtered'  
dataset_path = 'images'  # Relative path within the bucket
default_batch_size = 32
default_image_size = (224, 224)

# experiment, make sure to increment the number
models_list = ['EfficientNetB0', 'MobileNetV2', 'MobileNetV3Large', 'MobileNetV3Small', 'InceptionV3', 'EfficientNetB7']
run_experiments = {
    'experiment_number': 1, 
    'model_types': [0, 1, 2, 3], # leave out the two models with weird image sizes for now
    'number_of_stages': 1, 
    'stage1': {'epochs': 5, 'trainable': False, 'trainable_layers': None},
    'stage2': {'epochs': 2, 'trainable': True, 'trainable_layers': -2}
}


In [ ]:
def write_results_to_file(filename, start_time, end_time, model_name, batch_size, epochs, 
                          input_shape, training_accuracy, validate_accuracy, training_loss, validate_loss):
    start_time_str = start_time.strftime('%Y-%m-%d %H:%M:%S')
    end_time_str = end_time.strftime('%Y-%m-%d %H:%M:%S')
    line = f'{start_time_str},{end_time_str},{model_name},{batch_size},{epochs},{input_shape},{training_accuracy},' \
           f'{validate_accuracy},{training_loss},{validate_loss}\n'
    if not os.path.exists(filename):      # Check if the file exists, and add a header if it's new
        header = 'start_time,end_time,model_name,batch_size,epochs,input_shape,training_accuracy,validate_accuracy,training_loss,validate_loss\n'
        with open(filename, "w") as f:
            f.write(header + line)
    else:
        with open(filename, "a") as f: # or append to existing file
            f.write(line)
    print(f'experiment tracking updated')
    return

In [ ]:
def load_gcs_dataset(bucket_name, dataset_path, model_image_size, model_batch_size):
    dataset = None
    gcs_dataset_path = f"gs://{bucket_name}/{dataset_path}"
    try:
        dataset = keras.utils.image_dataset_from_directory(gcs_dataset_path, image_size=model_image_size,
            batch_size=model_batch_size, label_mode='categorical',)  # categorical is for softmax layer
    except Exception as e:
        print(f'error loading dataset from gcs: {e}')    
    return dataset


In [ ]:
# split into its own cell so we do not have to repeat this long operation, takes 10 minutes....
start_time = time.time()
start_time_datetime = datetime.datetime.fromtimestamp(start_time)
print(f'Start time: {start_time_datetime.strftime("%Y-%m-%d %H:%M:%S")}')

# need to load this each time for different batch and image size, load default of 224x224 with batch_size of 32
default_train_dataset = load_gcs_dataset(gcs_bucket, os.path.join(dataset_path, 'train'), default_image_size, default_batch_size)
default_validate_dataset = load_gcs_dataset(gcs_bucket, os.path.join(dataset_path, 'test'), default_image_size, default_batch_size)

if default_train_dataset is None or default_validate_dataset is None:
    print(f'dataset loading failed.')
    
num_classes = len(default_train_dataset.class_names) # get class count, same no mater how the data is loaded

In [ ]:
# use the experiments dictionary to train the models


start_time = time.time()
start_time_datetime = datetime.datetime.fromtimestamp(start_time)
print(f'Start time: {start_time_datetime.strftime("%Y-%m-%d %H:%M:%S")}')

# load model and modify head
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # unlock top layer for softmax replacement
base_model.trainable = True  # Unlock the base model, note this should be changed to freeze some layers per lit review ??

# add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Optional: Add more dense layers
predictions = Dense(num_classes, activation='softmax')(x)

# create the model with rescaling layer, this will automatically normalize the images within the model, bulletproofs feeder code
inputs = base_model.input
rescaled_inputs = tf.keras.layers.Rescaling(1./255)(inputs)  # normalize pixel values to [0, 1], not sure if this was done in the old mobilenetv2 model
x = base_model(rescaled_inputs) # pass the rescaled input through the base model
model = Model(inputs=inputs, outputs=predictions) # use the original inputs, this is weird, but how it was done in tutorial...

# compile and train model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',  # Important for multi-class classification
              metrics=['accuracy'])
history = model.fit(default_train_dataset, epochs=epochs, initial_epoch=0, validation_data=default_validate_dataset)  # initial epoch=0 is traing from scratch

model.save('mobilenet_retrained.h5')  # save the model

# plot results, none error here means model did not train
acc = history.history['accuracy']
validate_acc = history.history['val_accuracy']
loss = history.history['loss']
validate_loss = history.history['val_loss']

epochs_plt = range(1, len(acc) + 1)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_plt, acc, 'b', label='Training accuracy')
plt.plot(epochs_plt, validate_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_plt, loss, 'b', label='Training loss')
plt.plot(epochs_plt, validate_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# record training time
end_time = time.time()
end_time_datetime = datetime.datetime.fromtimestamp(end_time)
print(f'End time: {end_time_datetime.strftime("%Y-%m-%d %H:%M:%S")}')


In [ ]:
# sample predictions
# small_img = image.load_img(img_path, target_size=(224, 224))
# small_img_array = image.img_to_array(small_img)
# small_img_array = np.expand_dims(small_img_array, axis=0)
# small_img_array = preprocess_input(small_img_array)
# small_predictions = model_small.predict(small_img_array)
# decoded_small_predictions = decode_predictions(small_predictions, top=1)[0]
# print(decoded_small_predictions)

In [ ]:
### old dead code.... 

In [ ]:
# IMAGE_SIZE = (224, 224)
# BATCH_SIZE = 32
# train_dataset = load_gcs_dataset(GCS_BUCKET, os.path.join(DATASET_PATH, 'train'), IMAGE_SIZE, BATCH_SIZE)
# validation_dataset = load_gcs_dataset(GCS_BUCKET, os.path.join(DATASET_PATH, 'test'), IMAGE_SIZE, BATCH_SIZE)

# if train_dataset is None or validation_dataset is None:
#     print("Dataset loading failed. Exiting.")
#     exit()

# NUM_CLASSES = len(train_dataset.class_names) # Get class count.

In [ ]:
# Sample Code for MobilenetV2
##### MobileNetV2 training sample, use for comparision to old model in feeder
# start_time = time.time()
# start_time_datetime = datetime.datetime.fromtimestamp(start_time)
# print(f'Start time: {start_time_datetime.strftime("%Y-%m-%d %H:%M:%S")}')

# ###### load mobilenetv2 and modify head
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # unlock top layer for softmax replacement
# base_model.trainable = True  # Unlock the base model, note this should be changed to freeze some layers per lit review ??

# # add custom layers
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)  # Optional: Add more dense layers
# predictions = Dense(num_classes, activation='softmax')(x)

# # create the model with Rescaling layer
# inputs = base_model.input
# rescaled_inputs = tf.keras.layers.Rescaling(1./255)(inputs)  # normalize pixel values to [0, 1], not sure if this was done in the old mobilenetv2 model
# x = base_model(rescaled_inputs) # pass the rescaled input through the base model
# model = Model(inputs=inputs, outputs=predictions) # use the original inputs, this is weird, but how it was done in tutorial...

# # compile model
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
#               loss='categorical_crossentropy',  # Important for multi-class classification
#               metrics=['accuracy'])

# # train the model
# history = model.fit(train_dataset, epochs=epochs, initial_epoch=0, validation_data=validation_dataset)  # initial epoch=0 is traing from scratch

# # save model
# model.save('mobilenet_retrained.h5')

# # plot results, error here means model did not train
# acc = history.history['accuracy']
# test_acc = history.history['val_accuracy']
# loss = history.history['loss']
# test_loss = history.history['val_loss']

# epochs_plt = range(1, len(acc) + 1)
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(epochs_plt, acc, 'b', label='Training accuracy')
# plt.plot(epochs_plt, test_acc, 'r', label='Test accuracy')
# plt.title('Training and Testing Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(epochs_plt, loss, 'b', label='Training loss')
# plt.plot(epochs_plt, test_loss, 'r', label='Test loss')
# plt.title('Training and Test Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

# # record training time
# end_time = time.time()
# end_time_datetime = datetime.datetime.fromtimestamp(end_time)
# print(f'End time: {end_time_datetime.strftime("%Y-%m-%d %H:%M:%S")}')
